In [29]:
import requests
from bs4 import BeautifulSoup
import re

### 오늘 날짜

In [ ]:
import datetime
now = datetime.datetime.now()
day = now.strftime("%Y-%m-%d")
print(day)

### 뉴스 리스트 가져오기

In [126]:
#스포츠 메뉴의 구조가 달라 따로 구함
def sport(Sports_list):
    for i in range(3):
        params={
            'page':i+1,
            's_mcd':'0107'
        }
        resp = requests.get('https://www.ytn.co.kr/photo/photo_list.php',params=params)
        soup = BeautifulSoup(resp.text)
        #리스트
        sec_tag = soup.find("div", id="ytn_list_v2014")
        dl_tag=sec_tag.find_all("dl",class_='photo_list')
        
        for j in dl_tag:
            # 날짜
            date_tag = j.find('dd',class_='date')
            date_d = date_tag.text
            date = date_d.split(" ")
            date = date[0].replace("[","")


            if day == date:
                dt_tag =j.find_all('dt')
                a_tag = dt_tag[0].find("a")
                title = a_tag.text
                
                
                if title in Sports_list:
                    continue
                    
                Sports_list.append(title)

In [127]:
url = 'https://www.ytn.co.kr/news/news_list_0101.html'

resp = requests.get(url)
soup = BeautifulSoup(resp.content)
menu = soup.find("ul",id='sub_1')
li_tag = menu.find_all('li',recursive=False)

a=[]
politics_list=[]
economy_list=[]
social_list=[]
domestic_list=[]
International_list=[]
Science_list=[]
Sports_list=[]
culture_list =[]
newlist = [politics_list,economy_list,social_list,domestic_list,
           International_list,Science_list,culture_list]
#3,4,5,6,7,8,9,10,11
url2 = 'https://www.ytn.co.kr/news/news_list.php?'
for idx in range(3,11,1):
    #메뉴이동
    a_tag =li_tag[idx].find('a')
    move = a_tag.get('href')
    a = re.findall('\d+',move)
    
    if idx == 10:
        sport(Sports_list)
        break
          
    for i in range(3):
        params={
            'page':i+1,
            's_mcd':a[0]
        }
        resp = requests.get(url2,params=params)
        soup = BeautifulSoup(resp.text)

        #리스트
        sec_tag = soup.find("div", id="ytn_list_v2014")
        
        span_tag=sec_tag.find_all("dl", class_="news_list_v2014")

        for j in span_tag:
            # 날짜
            date_tag = j.find('dd',class_='date')
            date_d = date_tag.text
            date = date_d.split(" ")
            date = date[0].replace("[","")


            if day == date:
                dt_tag =j.find_all('dt')
                a_tag = dt_tag[0].find("a")
                title = a_tag.text
                
                
                if title in newlist[idx-3]:
                    continue
                    
                newlist[idx-3].append(title)
           

            



## 텍스트 파일로 저장

In [128]:
f = open('list/politics.txt','w')

for i in politics_list:
    msg = i
    f.write(i+"\n")
f.close()    

In [129]:
f = open('list/social.txt','w')

for i in social_list:
    msg = i
    f.write(i+"\n")
f.close()   

In [130]:
f = open('list/economy.txt','w')

for i in economy_list:
    msg = i
    f.write(i+"\n")
f.close()   

In [131]:
f = open('list/domestic.txt','w')

for i in domestic_list:
    msg = i
    f.write(i+"\n")
f.close()   

In [132]:
f = open('list/International.txt','w')

for i in International_list:
    msg = i
    f.write(i+"\n")
f.close()   

In [133]:
f = open('list/Science.txt','w')

for i in Science_list:
    msg = i
    f.write(i+"\n")
f.close()

In [134]:
f = open('list/culture.txt','w')

for i in culture_list:
    msg = i
    f.write(i+"\n")
f.close()   

In [123]:
f = open('list/Sports.txt','w')

for i in Sports_list:
    msg = i
    f.write(i+"\n")
f.close()   

# 명사 분리

In [138]:
from konlpy.tag import Twitter
from collections import Counter

In [139]:
menu_list=['list/politics.txt','list/social.txt','list/economy.txt','list/domestic.txt',
          'list/International.txt','list/Science.txt','list/culture.txt','list/Sports.txt']


def get_tags(text, ntags=50):
    spliter = Twitter()
    
    # konlpy의 Twitter객체
    nouns = spliter.nouns(text)
    
    # nouns 함수를 통해서 text에서 명사만 분리/추출
    count = Counter(nouns)
    
    
    # Counter객체를 생성하고 참조변수 nouns할당
    return_list = []  # 명사 빈도수 저장할 변수
    for n, c in count.most_common(ntags):
        temp = {'tag': n, 'count': c}
        return_list.append(temp)
        
        
    # most_common 메소드는 정수를 입력받아 객체 안의 명사중 빈도수
    # 큰 명사부터 순서대로 입력받은 정수 갯수만큼 저장되어있는 객체 반환
    # 명사와 사용된 갯수를 return_list에 저장합니다.
    return return_list

def main():
    for i in menu_list:
        text_file_name = i
        # 분석할 파일
        
        noun_count = 20
        # 최대 많은 빈도수 부터 20개 명사 추출
        
        
        output_file_name = i
        # count.txt 에 저장
        
        
        open_text_file = open(text_file_name, 'r',encoding='euc-kr')
        # 분석할 파일을 open 
        i = i.split("/")
        
        text = open_text_file.read() #파일을 읽습니다.
        tags = get_tags(text, noun_count) # get_tags 함수 실행
        open_text_file.close()   #파일 close
        open_output_file = open('list/count/' + i[1], 'w',encoding='euc-kr')
        # 결과로 쓰일 count.txt 열기
        for tag in tags:
            noun = tag['tag']
            count = tag['count']
            open_output_file.write('{} {}\n'.format(noun, count))
        # 결과 저장
        open_output_file.close() 
    
    
    
if __name__ == '__main__':
    main()

## 시각화

In [151]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

%matplotlib inline

import matplotlib
from IPython.display import set_matplotlib_formats
count_list=['list/count/politics.txt','list/count/social.txt','list/count/economy.txt','list/count/domestic.txt',
           'list/count/International.txt','list/count/Science.txt','list/count/culture.txt','list/count/Sports.txt']

for i in count_list:
    open_text_file = open(i, 'r',encoding='euc-kr')
    text = open_text_file.read()
    split1 = text.split("\n")
    for i in split1:
        split2 = i.split(" ")
        print(tuple(split2))
        
    

    font_path = 'C:/Windows/Fonts/malgun.ttf';
    wc = WordCloud(font_path = font_path, background_color = 'white', width=800, height=600)
    cloud = wc.generate_from_frequencies(dict(tags))
    plt.figure(figsize=(10,8))
    plt.axis('off')
    print(i)
    plt.imshow(cloud)

('부동산', '9')
('본회의', '8')
('국회', '8')
('통합', '7')
('공급', '7')
('법', '7')
('주택', '6')
('법안', '6')
('통과', '6')
('속보', '6')
('오늘', '6')
('민주당', '5')
('대책', '5')
('후속', '5')
('대통령', '5')
('점검', '5')
('정', '4')
('집중호우', '4')
('것', '3')
('처리', '3')
('',)


ValueError: dictionary update sequence element #0 has length 0; 2 is required